<a href="https://colab.research.google.com/github/Vicky-0222/NLP/blob/master/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Нейронные сети для преобразования текста**

Импорт необходимых библиотек

In [17]:
import numpy as np
import pandas as pd

### Создать нейронную сеть с нуля, т.е. не используя готовые библиотеки. Пример работы на любом табличном датасете.


Функция для линейной регрессии

In [18]:
def linear_regression(X: np.ndarray, weights: np.ndarray, bi) -> np.ndarray:
    return np.dot(X, weights) + bias

Функция для активации (сигмоид)

In [19]:
def activation_func(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))

Функция для нейрона

In [20]:
def neuron(X: np.ndarray, weights: np.ndarray, bias: float) -> np.ndarray:
    temp_result = linear_regression(X, weights, bias)
    result = activation_func(temp_result)
    return result

Пример работы

In [21]:
data = {
        'feature1': [0.1, 0.2, 0.3, 0.4, 0.5],
        'feature2': [0.5, 0.4, 0.3, 0.2, 0.1],
        'label': [0, 0, 1, 1, 1]
    }

df1 = pd.DataFrame(data)

In [22]:
X = df1[['feature1', 'feature2']].values # Входные признаки
y = df1['label'].values # Целевая переменная

In [23]:
weights = np.random.rand(X.shape[1]) # Генерация весов
bias = np.random.rand() # Генерация смещений

In [24]:
output = neuron(X, weights, bias)
print(output)

[0.65427153 0.64678983 0.63923551 0.63161163 0.62392143]


### Сделать класс, в котором реализована возможность задать количество нейронов в скрытом слое и провести обучение.


In [25]:
class NeuralNetwork:
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size, learning_rate=0.01):
        np.random.seed(42)
        self.lr = learning_rate

        # Инициализация весов и смещений
        self.W1 = np.random.randn(input_size, hidden1_size) * 0.1
        self.b1 = np.zeros((1, hidden1_size))

        self.W2 = np.random.randn(hidden1_size, hidden2_size) * 0.1
        self.b2 = np.zeros((1, hidden2_size))

        self.W3 = np.random.randn(hidden2_size, output_size) * 0.1
        self.b3 = np.zeros((1, output_size))

    def activation_func(self, x):
        return 1 / (1 + np.exp(-x))  # Сигмоида

    def activation_func_derivative(self, x):
        return x * (1 - x)  # Производная сигмоиды

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.activation_func(self.z1)

        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.activation_func(self.z2)

        self.z3 = np.dot(self.a2, self.W3) + self.b3
        self.a3 = self.activation_func(self.z3)

        return self.a3

    def backward(self, X, y):
        output_error = self.a3 - y  # Ошибка на выходе
        output_delta = output_error * self.activation_func_derivative(self.a3)

        hidden2_error = output_delta.dot(self.W3.T)
        hidden2_delta = hidden2_error * self.activation_func_derivative(self.a2)

        hidden1_error = hidden2_delta.dot(self.W2.T)
        hidden1_delta = hidden1_error * self.activation_func_derivative(self.a1)

        # Обновление весов и смещений
        self.W3 -= self.a2.T.dot(output_delta) * self.lr
        self.b3 -= np.sum(output_delta, axis=0, keepdims=True) * self.lr

        self.W2 -= self.a1.T.dot(hidden2_delta) * self.lr
        self.b2 -= np.sum(hidden2_delta, axis=0, keepdims=True) * self.lr

        self.W1 -= X.T.dot(hidden1_delta) * self.lr
        self.b1 -= np.sum(hidden1_delta, axis=0, keepdims=True) * self.lr

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            self.forward(X)
            self.backward(X, y)
            if epoch % 100 == 0:
                loss = np.mean(np.square(y - self.a3))  # Среднеквадратичная ошибка
                print(f'Epoch {epoch}, Loss: {loss}')

Пример работы

In [26]:
# Подготовка данных
df2= pd.DataFrame(data)

X = df2[['feature1', 'feature2']].values
y = df2['label'].values.reshape(-1, 1)  # Преобразуем y в столбец

# Создаем и обучаем нейронную сеть
nn = NeuralNetwork(input_size=2, hidden1_size=3, hidden2_size=3, output_size=1, learning_rate=0.1)
nn.train(X, y, epochs=1000)

# Прогоняем данные через нейронную сеть
output = nn.forward(X)
print("Выход нейронной сети:", output)

Epoch 0, Loss: 0.25343579202470934
Epoch 100, Loss: 0.2400066653815962
Epoch 200, Loss: 0.24000605090893457
Epoch 300, Loss: 0.24000581026249956
Epoch 400, Loss: 0.24000557680809279
Epoch 500, Loss: 0.240005350185569
Epoch 600, Loss: 0.24000513006163615
Epoch 700, Loss: 0.24000491611605518
Epoch 800, Loss: 0.24000470804081556
Epoch 900, Loss: 0.24000450553935856
Выход нейронной сети: [[0.59996788]
 [0.59996429]
 [0.5999607 ]
 [0.59995711]
 [0.59995352]]


### Реализовать GPT как в п.2

In [27]:
class Head:
    def __init__(self, n_embd, head_size, learning_rate=0.01):
        self.n_embd = n_embd
        self.head_size = head_size
        self.lr = learning_rate

        # Инициализация весов для key, query, value
        self.key_weights = np.random.randn(n_embd, head_size) * 0.01
        self.query_weights = np.random.randn(n_embd, head_size) * 0.01
        self.value_weights = np.random.randn(n_embd, head_size) * 0.01
        # bias - False

        #создаем маску
    def create_mask(self, T):
        mask = np.tril(np.ones((T, T)), k=0)  # (T, T)
        return mask

    def forward(self, x):
        # x имеет размер (batch, time-step, channels)
        B, T, C = x.shape

        k = np.dot(x.reshape(-1, C), self.key_weights).reshape(B, T, -1)  # (B, T, head_size)
        q = np.dot(x.reshape(-1, C), self.query_weights).reshape(B, T, -1)  # (B, T, head_size)
        v = np.dot(x.reshape(-1, C), self.value_weights).reshape(B, T, -1)  # (B, T, head_size)

        # Вычисляем оценки внимания (affinities)
        wei = np.matmul(q, k.transpose(0, 2, 1)) * (k.shape[-1] ** -0.5  )  # (B, T, head_size) @ (B, head_size, T) -> (B, T, T)

        # Применяем маску
        mask = self.create_mask(T)  # (T, T)
        wei = np.where(mask == 0, -np.inf, wei)

        # Применяем softmax для получения весов внимания
        wei = np.exp(wei - np.max(wei, axis=-1, keepdims=True))
        wei /= np.sum(wei, axis=-1, keepdims=True)  # Нормализация

        # Выполняем взвешенное агрегирование value
        out = np.matmul(wei, v)  # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)
        return out

    def loss_derivative(self, targets):
        return 2 * (self.v - targets) / targets.size #среднеквадратичная ошибка

    def backward(self, x, targets):
        B, T, C = x.shape

        # Вычисление градиента потерь
        d_out = self.loss_derivative(targets)

        # Обратное распространение через внимание
        d_v = np.matmul(d_out, self.wei)  # Градиент для value
        d_wei = np.matmul(self.q.transpose(0, 2, 1), d_out)  # Градиенты для весов внимания

        # Обратное распространение через key и query
        d_q = np.matmul(d_wei, self.k)  # Градиенты для query
        d_k = np.matmul(d_wei.transpose(0, 2, 1), self.q)  # Градиенты для key

        # Обновление весов
        self.key_weights -= self.lr * np.dot(x.reshape(-1, C).T, d_k.reshape(-1, self.head_size))
        self.query_weights -= self.lr * np.dot(x.reshape(-1, C).T, d_q.reshape(-1, self.head_size))
        self.value_weights -= self.lr * np.dot(x.reshape(-1, C).T, d_v.reshape(-1, self.head_size))


        return np.mean(d_out)  # Возвращаем среднюю потерю

        # Обучение
    def train(self, data, targets, epochs):
        for epoch in range(epochs):
            total_loss = 0
            for x, y in zip(data, targets):
                loss = self.backward(x, y)
                total_loss += loss
            print(f'Epoch {epoch + 1}, Loss: {total_loss / len(data):.4f}')


Пример работы

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Пример текстовых данных
texts = [
    "hello world",
    "hello there",
    "world of AI",
    "AI is the future"
]

# Преобразование текстов в TF-IDF векторы
vectorizer = TfidfVectorizer()
tfidf_data = vectorizer.fit_transform(texts).toarray()

# Параметры модели
n_embd = tfidf_data.shape[1]  # Размерность векторного представления (количество уникальных слов)
head_size = 2  # Размер головы внимания
learning_rate = 0.01  # Скорость обучения
epochs = 10  # Количество эпох

# Создаем экземпляр класса Head
head = Head(n_embd=n_embd, head_size=head_size, learning_rate=learning_rate)

# Генерируем случайные цели для обучения (например, сдвинутые на один вектор)
targets = np.roll(tfidf_data, -1, axis=0)

# Обучаем модель
head.train(tfidf_data, targets, epochs)